In [1]:
import re
import sys

sys.path.append("..")
from rocket_rag.utils import *
from rocket_rag.node import Node
from rocket_rag.node_indexing import NodeIndexer

from typing import List, Any, Dict, Tuple

from pyts.transformation import ROCKET
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score

c:\Users\HAOXUAN\miniconda3\envs\agents\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class BaseVectorStore():
    """Simple custom Vector Store.

    Stores documents in a simple in-memory dict.

    """

    stores_text: bool = True

    def get(self, text_id: str) -> List[float]:
        """Get vector for a text ID."""
        pass

    def add(self, nodes: List[Node]) -> List[str]:
        """Add nodes to index"""
        pass

    def delete(self, node_id: str, **delete_kwargs: Any) -> None:
        """Delete nodes using node_id."""
        pass

    def query(self, query: str, **kwargs: Any):
        """Get nodes for response"""
        pass 

In [3]:
class VectorStore(BaseVectorStore):
    """An updated version of above SimpleVectorStore"""

    stores_text: bool = True

    def __init__(self) -> None:
        """Init params"""
        self.node_dict: Dict[str, Node] = {}
        self.nodes = self.node_dict.values()
        super().__init__()

    def get(self, text_id: str) -> List[float]:
        """Get vector for a text ID."""
        return self.node_dict[text_id]
    
    def add(self, nodes: List[Node]) -> List[str]:
        """Add nodes to index"""
        for node in nodes:
            self.node_dict[node.node_id] = node
    
    def delete(self, node_id: str, **delete_kwargs: Any) -> None:
        """Delete nodes using node_id"""
        del self.node_dict[node_id]

In [4]:
loguru.logger.debug(f'Testing on vector store module...')
load = '-40kg'
node_indexer = NodeIndexer()
nodes = node_indexer.load_node_indexing(f'../store/nodes_{load}.pkl')

2024-05-25 13:31:52.785 | DEBUG    | __main__:<module>:1 - Testing on vector store module...
2024-05-25 13:31:52.786 | DEBUG    | rocket_rag.node_indexing:load_node_indexing:98 - Loading all nodes...
2024-05-25 13:31:53.198 | INFO     | rocket_rag.node_indexing:load_node_indexing:102 - All nodes are loaded.


In [5]:
loguru.logger.debug(f'Initializing vector store...')
vector_store = VectorStore()
vector_store.add(nodes)
loguru.logger.info(f'Loaded nodes into the vector store.')

2024-05-25 13:31:54.817 | DEBUG    | __main__:<module>:1 - Initializing vector store...
2024-05-25 13:31:54.817 | INFO     | __main__:<module>:4 - Loaded nodes into the vector store.


In [6]:
rocket_features = np.array([node.get_rocket_feature() for node in vector_store.nodes])
doc_ids = np.array([node.id_ for node in vector_store.nodes])

In [7]:
if_files_dict = parse_files(main_directory=INFERENCE_DIR)
if_ts_files = if_files_dict[load]

In [43]:
# np.random.seed(42)
rand_idx = np.random.randint(0, len(if_ts_files))
if_ts_filename = if_ts_files[rand_idx]
print(f'Randomly selected file: {if_ts_filename}')

if_rocket_feature = fit_transform(ts_filename=[if_ts_filename],
                                  field='current',
                                  smooth=True,
                                  smooth_ws=15,
                                  tolist=False,
                                  verbo=True)

2024-05-25 14:24:23.633 | DEBUG    | rocket_rag.utils:fit_transform:149 - Extract the time series data points
2024-05-25 14:24:23.657 | INFO     | rocket_rag.utils:fit:125 - time series extracted SUCCESSFULLY.
2024-05-25 14:24:23.658 | DEBUG    | rocket_rag.utils:fit_transform:161 - Rocket transforming...


Randomly selected file: ../data/inference/-40kg\spalling5\spalling5_-40_3_2.csv


### Use sklearn to do the retrieval

In [23]:
def parse_result(res: Union[List[Any], np.ndarray]):
    tar_str = res.tolist()[0] if isinstance(res, np.ndarray) else res
    return re.match(r'(.*?)_', tar_str).group(1)

In [95]:
knn = KNeighborsClassifier(n_neighbors=1, weights='distance', metric='euclidean')
knn.fit(rocket_features, doc_ids)
knn_pred = knn.predict(if_rocket_feature)
dist, ids = knn.kneighbors(if_rocket_feature, n_neighbors=5, return_distance=True)
ids_to_doc = {i: doc_ids[i] for i in range(len(doc_ids))}
print(dist.squeeze().tolist())
# print(ids_to_doc[ids.squeeze().tolist()])
print([ids_to_doc[i] for i in ids.tolist()[0]])
# print(parse_result(knn_pred))

[2.913731003851641, 3.2295925336847713, 3.265172098148915, 3.329911209755798, 3.5833409196557535]
['spalling5_-40_3_4', 'spalling6_-40_4_3', 'spalling4_-40_5_4', 'spalling4_-40_2_1', 'spalling4_-40_6_2']


In [20]:
ridge = RidgeClassifier()
ridge.fit(rocket_features, doc_ids)
ridge_pred = ridge.predict(if_rocket_feature)
print(parse_result(ridge_pred))

backlash2


### Use handcraft coding for KNN retrieval

In [22]:
euclidean = [np.linalg.norm(rf - if_rocket_feature.squeeze()) for rf in rocket_features]
cosine = [np.dot(rf, if_rocket_feature.squeeze()) / (np.linalg.norm(rf) * np.linalg.norm(if_rocket_feature.squeeze())) for rf in rocket_features]

In [23]:
sorted([(euclidean[i], doc_ids[i]) for i in range(len(doc_ids))], key=lambda x: x[0])[:3]

[(7.6432402002574165, 'backlash2_-40_8_4'),
 (9.13494425676515, 'backlash2_-40_7_2'),
 (9.529327858850607, 'backlash2_-40_9_5')]

In [24]:
sorted([(cosine[i], doc_ids[i]) for i in range(len(doc_ids))], key=lambda x: x[0], reverse=True)[:3]

[(0.9988390402632548, 'backlash2_-40_8_4'),
 (0.9986860760662098, 'backlash2_-40_9_5'),
 (0.9985502045686941, 'backlash2_-40_7_2')]